Import dependencies

In [9]:
import landmark_detector as ld
import os
import random
import numpy as np
import keras
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras import layers

Set variables

In [13]:
words = ['deaf', 'eat', 'fish', 'friend', 'like', 'milk', 'nice', 'no', 'orange', 'teacher', 'want', 'what', 'where', 'yes']
select_words = ['no', 'eat', 'teacher', 'want', 'fish']
path = '../preprocessing/dataset/test'
num_features = 126
model_name = 'draft_model'
fps = 20

Get data from mediapipe


In [14]:
def get_data(words, path, detector_path):
    detector = ld.get_detector(detector_path)

    X = []
    y = []

    num_videos = 0
    highest_frame = 0

    bad_videos = 0

    for word in tqdm(words):
        word_path = os.path.join(path, word)
        
        video_files = [f for f in os.listdir(word_path) if f.endswith('.mp4')]
        
        for video_file in tqdm(video_files, desc=word):
            video_path = os.path.join(word_path, video_file)
            
            try:
                video_X = []
                landmarks, current_frames = ld.get_landmarks(video_path, detector)
                
                if len(landmarks) == 0:
                    bad_videos+=1
                    continue
                
                if current_frames > highest_frame:
                    highest_frame = current_frames
                
                for frame in range(len(landmarks)):
                    features = np.array(landmarks[frame]).flatten()
                    video_X.append(features)
                
                X.append(video_X)
                y.append(words.index(word))
                num_videos += 1

            except Exception as e:
                print(f"Error processing video {video_file}: {e}")
                continue 

    return X, y, num_videos, highest_frame, bad_videos

X, y, num_videos, highest_frame, bad_videos = get_data(select_words, path, '../models/hand_landmarker.task')

print('Number of videos:', num_videos)
print('Highest frame:', highest_frame)
print('Videos with no landmarkers detected: ', bad_videos)







































no: 100%|██████████| 8/8 [00:02<00:00,  2.82it/s]

























eat: 100%|██████████| 5/5 [00:02<00:00,  2.23it/s]













































teacher: 100%|██████████| 10/10 [00:04<00:00,  2.06it/s]





























want: 100%|██████████| 6/6 [00:02<00:00,  2.16it/s]





















fish: 100%|██████████| 4/4 [00:02<00:00,  1.96it/s]

100%|██████████| 5/5 [00:14<00:00,  2.96s/it]

Number of videos: 22
Highest frame: 20
Videos with no landmarkers detected:  11


Padding and Masking X

In [15]:
def padX(X, num_videos, highest_frame, num_features):
    padded_X = np.zeros((num_videos, highest_frame, num_features))
    mask = np.ones((num_videos, highest_frame, num_features)) 
    for i in range(num_videos):
        video = X[i]
        for j in range(len(video)):
            frame = video[j]
            if len(frame) < num_features:
                padded_X[i, j, :] = np.pad(frame, (0, num_features - len(frame)), 'constant')
                mask[i, j, len(frame):] = 0
            else:
                padded_X[i, j, :] = frame
        if len(video) < highest_frame:
            mask[i, len(video):, :] = 0

    return padded_X, mask

padded_X, mask = padX(X, num_videos, highest_frame, num_features)
print(padded_X.shape)

(22, 20, 126)


Create model

In [16]:
model = keras.Sequential()

model.add(keras.Input(shape=(highest_frame, num_features)))
model.add(layers.Masking(mask_value=0.0))
model.add(layers.LSTM(64))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(select_words), activation='sigmoid'))


model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 20, 126)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,141 (199.77 KB)

 Trainable params: 51,141 (199.77 KB)

 Non-trainable params: 0 (0.00 B)

Train model

In [18]:
X_train, X_temp, y_train, y_temp = train_test_split(padded_X, y, test_size=0.3, random_state=42) 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) 


X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

model.save(f'../models/{model_name}.keras')

with open(f"../models/{model_name}.env", "w") as file:
    file.write(f"MAX_FRAMES={highest_frame}\n")
    file.write(f"NUM_FEATURES={num_features}\n")
    file.write(f"WORDS={','.join(select_words)}\n")
    file.write(f"FPS={fps}\n")

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.2000 - loss: 1.6611 - val_accuracy: 0.0000e+00 - val_loss: 1.6950
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.2000 - loss: 1.5711 - val_accuracy: 0.0000e+00 - val_loss: 1.7031
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.2000 - loss: 1.6023 - val_accuracy: 0.0000e+00 - val_loss: 1.7053
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.1333 - loss: 1.5664 - val_accuracy: 0.0000e+00 - val_loss: 1.7216
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.4667 - loss: 1.5036 - val_accuracy: 0.0000e+00 - val_loss: 1.7511
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.4000 - loss: 1.4343 - val_accuracy: 0.0000e+00 - val_loss: 1.7793
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.3333 - loss: 1.5113 - val_accuracy: 0.0000e+00 - val_loss: 1.8027
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.2000 - loss: 1.5079 - val_acc

Evaluate model

In [19]:
results = model.evaluate(X_test, y_test)

print('Test loss:', results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.2500 - loss: 1.5577
Test loss: [1.5577479600906372, 0.25]


Use model

In [ ]:
i = random.randint(0,X_test.shape[0]-1)


X_prediction = X_test[i,:,:]
y_prediction = select_words[y_test[i]]

print(model.predict(np.array([X_prediction])))
print("should be", y_prediction)
print("predicted", select_words[np.argmax(model.predict(np.array([X_prediction])))])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
[[0.3319136  0.4899597  0.63422    0.64276433 0.43016696]]
should be teacher
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
predicted want


: 